In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Input, concatenate
from tensorflow.keras.models import Model


In [ ]:
file_path = 'steam.csv'
data = pd.read_csv(file_path)
data.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [ ]:
data = data[['name', 'categories', 'genres', 'price', 'positive_ratings', 'negative_ratings']]
data['user_id'] = np.random.randint(1, 1000, size=len(data))
data['rating'] = np.clip(data['positive_ratings'] - data['negative_ratings'], 0, 10)


<ipython-input-11-b420d2b2e980>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['user_id'] = np.random.randint(1, 1000, size=len(data))
<ipython-input-11-b420d2b2e980>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating'] = np.clip(data['positive_ratings'] - data['negative_ratings'], 0, 10)


In [ ]:
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader, Dataset

reader = Reader(rating_scale=(0, 10))

dataset = Dataset.load_from_df(data[['user_id', 'name', 'rating']], reader)

trainset, testset = train_test_split(dataset, test_size=0.2)

model = SVD(n_epochs=100, verbose=True)
model.fit(trainset)

predictions = model.test(testset)
print("RMSE (Root Mean Squared Error):", accuracy.rmse(predictions))


Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [15]:
def recommend_based_on_inputs(user_id, category, genre, max_price, model, data, n=5):
    all_games = data['name'].unique()
    user_games = data[data['user_id'] == user_id]['name'].unique()
    games_to_predict = [game for game in all_games if game not in user_games]

    predictions = [
        (game, model.predict(user_id, game).est)
        for game in games_to_predict
        if category.lower() in data[data['name'] == game]['categories'].iloc[0].lower()
        and genre.lower() in data[data['name'] == game]['genres'].iloc[0].lower()
        and data[data['name'] == game]['price'].iloc[0] <= max_price
    ]

    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]


In [17]:
user_id = int(input("Enter your user ID: "))
category = input("Enter preferred category: ")
genre = input("Enter preferred genre: ")
max_price = float(input("Enter maximum price: "))

all_games = data['name'].unique()
user_games = data[data['user_id'] == user_id]['name'].unique()
games_to_predict = [game for game in all_games if game not in user_games]

predictions = [
    (game, model.predict(user_id, game).est)
    for game in games_to_predict
    if category.lower() in data[data['name'] == game]['categories'].iloc[0].lower()
    and genre.lower() in data[data['name'] == game]['genres'].iloc[0].lower()
    and data[data['name'] == game]['price'].iloc[0] <= max_price
]

recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:5]

print(f"\nTop 5 recommendations for User {user_id}:")
for game, rating in recommendations:
    game_data = data[data['name'] == game].iloc[0]
    print(f"{game} - Genre: {game_data['genres']} - Price: ${game_data['price']:.2f} - Predicted Rating: {rating:.2f}")



Enter your user ID: 2
Enter preferred category: single-player
Enter preferred genre: action
Enter maximum price: 3.00

Top 5 recommendations for User 2:
The Construct - Genre: Action;Adventure;Indie - Price: $1.99 - Predicted Rating: 9.43
ORION: Prelude - Genre: Action;Adventure;Indie;RPG - Price: $0.79 - Predicted Rating: 9.40
The Ship: Murder Party - Genre: Action;Indie;RPG - Price: $0.00 - Predicted Rating: 9.38
Deep Dark Fight - Genre: Nudity;Action;Adventure;Indie - Price: $2.09 - Predicted Rating: 9.30
Super Crome: Bullet Purgatory - Genre: Action;Adventure;Early Access - Price: $1.69 - Predicted Rating: 9.25
